## Inserting Data using Stage Table

Let us understand how to insert data into order_items with Parquet file format. 

In [5]:
%%HTML
<iframe width="560" height="315" src="https://www.youtube.com/embed/qb869zZKZk8?rel=0&amp;controls=1&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>

As data is in text file format and our table is created with Parquet file format, we will not be able to use LOAD command to load the data.

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/order_items'
    OVERWRITE INTO TABLE order_items

* Above load command will be successful, however when we try to query it will fail as the query expects data to be in Parquet file format.

In [ ]:
%%sql

SELECT * FROM order_items LIMIT 10

In [ ]:
%%sql

TRUNCATE TABLE order_items

Following are the steps to get data into table which is created using different file format or delimiter than our source data.

* We need to create stage table with text file format and comma as delimiter (order_items_stage).
* Load data from our files in local file system to stage table.
* Using stage table run insert command to insert data into our target table (order_items).

Let us see an example of inserting data into the target table from staging table.

In [ ]:
%%sql

USE itversity_retail

In [ ]:
%%sql

SHOW tables

In [ ]:
%%sql

CREATE TABLE order_items_stage (
  order_item_id INT,
  order_item_order_id INT,
  order_item_product_id INT,
  order_item_quantity INT,
  order_item_subtotal FLOAT,
  order_item_product_price FLOAT
) ROW FORMAT DELIMITED FIELDS TERMINATED BY ','

In [ ]:
spark.sql("DESCRIBE FORMATTED order_items_stage").show(200, false)

In [ ]:
%%sql

LOAD DATA LOCAL INPATH '/data/retail_db/order_items' INTO TABLE order_items_stage

In [ ]:
%%sql

SELECT * FROM order_items_stage LIMIT 10

In [ ]:
%%sql

TRUNCATE TABLE order_items

In [ ]:
%%sql

INSERT INTO TABLE order_items
SELECT * FROM order_items_stage

In [ ]:
%%sql

SELECT * FROM order_items LIMIT 10

In [ ]:
%%sql

SELECT count(1) FROM order_items

* `INSERT INTO` will append data into the target table by adding new files.

In [ ]:
%%sql

INSERT INTO TABLE order_items
SELECT * FROM order_items_stage

In [ ]:
%%sql

SELECT * FROM order_items LIMIT 10

In [ ]:
%%sql

SELECT count(1) FROM order_items

* `INSERT OVERWRITE` will overwrite the data in target table by deleting the files related to old data from the directory pointed by the Spark Metastore table.

In [ ]:
%%sql

INSERT OVERWRITE TABLE order_items
SELECT * FROM order_items_stage

In [ ]:
%%sql

SELECT * FROM order_items

In [ ]:
%%sql

SELECT count(1) FROM order_items

In [ ]:
import sys.process._

s"hdfs dfs -ls /user/${username}/warehouse/${username}_retail.db/order_items" !